In [1]:
import torch
import sys
import os
import lightning as L
from lightning.pytorch.callbacks import ModelCheckpoint, EarlyStopping
from lightning.pytorch.loggers import MLFlowLogger

project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))


if project_root not in sys.path:
    sys.path.append(project_root)
    
from model_utils import ModelKANCNN, ClassificationData, no_augmentation

torch.cuda.empty_cache()
torch.set_float32_matmul_precision("medium")
L.seed_everything(111)

Seed set to 111


111

In [2]:
example_hyperparams = {
    "learning_rate": 0.1,
    "batch_size": 128,
    "weight_decay": 0.001,
    "dropout": 0.2,
}
model = ModelKANCNN(batch_size=example_hyperparams["batch_size"], learning_rate=example_hyperparams["learning_rate"], weight_decay=example_hyperparams["weight_decay"], dropout=example_hyperparams["dropout"])
logger = MLFlowLogger(save_dir="mlruns", experiment_name="KAN-CNN")
early_stop = EarlyStopping(monitor="train_loss", patience=3, mode="min", verbose=True, min_delta=0.01)
checkpoint_callback = ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/kan", filename=f"lr={model.hyperparameters["learning_rate"]}_bs={model.hyperparameters["batch_size"]}_wd={model.hyperparameters["weight_decay"]}_dropout={model.hyperparameters["dropout"]}" + "-{epoch:02d}-{val_f1_macro:.2f}")
trainer = L.Trainer(max_epochs=20, logger=logger, num_sanity_val_steps=0, enable_model_summary=False, deterministic=False, callbacks=[early_stop, checkpoint_callback], precision="16-mixed")
data = ClassificationData(batch_size=model.hyperparameters["batch_size"])
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)

TypeError: KAN_Convolutional_Layer.__init__() got an unexpected keyword argument 'n_convs'

### No Data Augmentation

In [ ]:
model = ModelKANCNN(example_hyperparams)
logger = MLFlowLogger(save_dir="mlruns", experiment_name="KAN-CNN-no-augmentation")
early_stop = EarlyStopping(monitor="train_loss", patience=3, mode="min", verbose=True, min_delta=0.01)
checkpoint_callback = ModelCheckpoint(monitor="val_f1_macro", mode="max", dirpath="checkpoints/kan", filename=f"no-augment_lr={model.hyperparameters["learning_rate"]}_bs={model.hyperparameters["batch_size"]}_wd={model.hyperparameters["weight_decay"]}_dropout={model.hyperparameters["dropout"]}" + "-{epoch:02d}-{val_f1_macro:.2f}")
trainer = L.Trainer(max_epochs=20, logger=logger, num_sanity_val_steps=0, enable_model_summary=False, deterministic=False, callbacks=[early_stop, checkpoint_callback], precision="16-mixed")
data = ClassificationData(batch_size=model.hyperparameters["batch_size"], transform=no_augmentation)
trainer.fit(model, datamodule=data)
trainer.test(model, datamodule=data)